In [13]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn import preprocessing

In [14]:
model = RandomForestClassifier(
            n_estimators = 200, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            max_depth= None, ## 개별 나무의 최대 깊이
            max_features= 5,
            random_state=100)

In [15]:
raw_data = pd.read_csv('./dataset/train.csv')
data = raw_data.copy()

In [16]:
data= data.drop(columns=['father','mother','gender','id'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   trait   262 non-null    int64 
 1   SNP_01  262 non-null    object
 2   SNP_02  262 non-null    object
 3   SNP_03  262 non-null    object
 4   SNP_04  262 non-null    object
 5   SNP_05  262 non-null    object
 6   SNP_06  262 non-null    object
 7   SNP_07  262 non-null    object
 8   SNP_08  262 non-null    object
 9   SNP_09  262 non-null    object
 10  SNP_10  262 non-null    object
 11  SNP_11  262 non-null    object
 12  SNP_12  262 non-null    object
 13  SNP_13  262 non-null    object
 14  SNP_14  262 non-null    object
 15  SNP_15  262 non-null    object
 16  class   262 non-null    object
dtypes: int64(1), object(16)
memory usage: 34.9+ KB


In [17]:
x_data = data.iloc[:,0:16]
y_data = data.iloc[:,16:17]
x_test = pd.read_csv('./dataset/test.csv')
x_test = x_test.drop(columns = ['id','father','mother','gender'])
y_data.head()

,class
0,B
1,C
2,B
3,A
4,C


In [18]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [19]:
snp_data = []
for col in snp_col:
    snp_data += list(x_data[col].values)
y_data = class_le.fit_transform(y_data)
snp_le.fit(snp_data)

for col in x_data.columns:
    if col in snp_col:
        x_data[col] = snp_le.transform(x_data[col])
        x_test[col] = snp_le.transform(x_test[col])

/Users/imbot/miniforge3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
model.fit(x_data,y_data)
preds = model.predict(x_test)

In [21]:
submit = pd.read_csv('./dataset/sample_submission.csv')
submit['class'] = class_le.inverse_transform(preds)
submit.to_csv('./dataset/submit.csv', index=False)